In [1]:
# measures ocr accuracy of zipped files

In [1]:
# get wordlists
with open('/Users/e/Documents/LiteraryLab/word lists/oed_wordlist.txt') as f:
    f = f.read().lower()
    f = set(f.split('\n'))

In [2]:
# compare with nltk

with open('/Users/e/nltk_data/corpora/words/en') as g:
    g = g.read().lower()
    g = set(g.split('\n'))

In [11]:
nltk_oed = list((set(g) & set(f)))
# element 0 is a space, so removing
nltk_oed.remove('')
nltk_oed = set(nltk_oed)

In [12]:
# get filenames from tar archive
def get_members(tar_path):
    import tarfile
    tf = tarfile.open(tar_path)
    tf_members = tf.getmembers()
    valid = []
    for i in tf_members:
        if i.isfile():
            valid.append(i)
    return(tf, valid)

In [14]:
def ocr_accuracy(text, wordlist = nltk_oed):
    from collections import Counter
    import os
    import re
    
    wc = 0
    d = {}
    f = text.lower()
    f = re.findall("[A-Z]{2,}(?![a-z])|[A-Z][a-z]+(?=[A-Z])|[\'\w\-]+", f)
    c = Counter(f)
    full_count = sum(c.values())
    valid = list(set(f) & wordlist)
    for el in set(c.elements()):
        if el in valid:
            wc += int(c[el])
#             d[el] = c[el] # uncomment to make a valid dtm
    d['_wordlist'] = 'nltk_oed'
    d['_ocr_accuracy'] = round(wc / full_count, 2)
    d['_word_count'] = full_count
    d['_valid_words'] = wc
    return(d)

In [15]:
def get_text(tf, filenames, wordlist):
    from bs4 import BeautifulSoup
    import pandas as pd
    
    output_list = []
    counter = 0
    
    for i, file in enumerate(filenames):
        with tf.extractfile(file) as f:
            output = {}
            fn = str(file).split('\'')[1]
            output['_fn'] = fn
            if f is None:
                continue
            xml = f.read()
            # get metadata
            soup = BeautifulSoup(xml, 'lxml')
            meta_tags = ['doctitle', 'arttyp', 'pmtitle', 'pcdta', 'pmq', 'word-count', 'auth']
            for tag in meta_tags:
                t = soup.find(tag)
                if t is None:
                    output['_' + tag] = 'NULL'
                    continue
                output['_' + t.name] = t.text
            # get ocr accuracy
            text = soup.text
            ocr = ocr_accuracy(text)
            # attach everything to dictionary
            output.update(ocr)
            output_list.append(output)
        print('\r',round(i/len(filenames)*100, 1),'%',end='')
            
            
    return(output_list)

In [16]:
def news_ocr(file):
    import os
    tf, valid = get_members(file)
    ld = get_text(tf, valid, wordlist = nltk_oed)
    for d in ld:
        d['_archive'] = os.path.split(file)[1]
    return(ld)

In [22]:
# path = '/Users/e/Documents/LiteraryLab/celebrity/xml_samples/bfexml_113712_1886_20110502.tgz'

In [23]:
# a = news_ocr(path)

slingshot -sling celebrity_ocr_accuracy.ipynb -stone news_ocr -path /Users/e/Documents/LiteraryLab/celebrity/xml_samples -ext tgz -savedir results_slingshot/celebrity_ocr_accuracy__news_ocr -parallel 16

In [21]:
# a[0]

{'_fn': './113712/1886/2360332/319663502.xml',
 '_doctitle': 'LYING IN STATE.',
 '_arttyp': 'article',
 '_pmtitle': 'Los Angeles Times (1886-1922)',
 '_pcdta': 'Dec 31, 1886',
 '_pmq': 'Los Angeles, Calif.',
 '_word-count': '590',
 '_auth': 'Telegraph to The Times',
 '_wordlist': 'nltk_oed',
 '_ocr_accuracy': 0.7,
 '_word_count': 762,
 '_valid_words': 536,
 '_archive': 'bfexml_113712_1886_20110502.tgz'}

# Postprocessing

In [25]:
import sys
sys.path.insert(0,'/Users/e/code/mpi-slingshot/')

In [28]:
l = []
from mpi_slingshot import stream_results
for path, data in stream_results('results_slingshot/celebrity_ocr_accuracy__news_ocr/cache/'):
    for d in data:
        l.append(d)

>> streaming: results_slingshot/celebrity_ocr_accuracy__news_ocr/cache/results.rank=0000.jsonl ...
>> streaming: results_slingshot/celebrity_ocr_accuracy__news_ocr/cache/results.rank=0001.jsonl ...
>> streaming: results_slingshot/celebrity_ocr_accuracy__news_ocr/cache/results.rank=0002.jsonl ...
>> streaming: results_slingshot/celebrity_ocr_accuracy__news_ocr/cache/results.rank=0003.jsonl ...
>> streaming: results_slingshot/celebrity_ocr_accuracy__news_ocr/cache/results.rank=0004.jsonl ...
>> streaming: results_slingshot/celebrity_ocr_accuracy__news_ocr/cache/results.rank=0005.jsonl ...
>> streaming: results_slingshot/celebrity_ocr_accuracy__news_ocr/cache/results.rank=0006.jsonl ...
>> streaming: results_slingshot/celebrity_ocr_accuracy__news_ocr/cache/results.rank=0007.jsonl ...
>> streaming: results_slingshot/celebrity_ocr_accuracy__news_ocr/cache/results.rank=0008.jsonl ...
>> streaming: results_slingshot/celebrity_ocr_accuracy__news_ocr/cache/results.rank=0009.jsonl ...
>> streami

In [30]:
import pandas as pd
df = pd.DataFrame(l)

In [36]:
df.shape

(11617, 13)

In [43]:
df.query('_ocr_accuracy >= 0.9').shape

(1, 13)